In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengolib
import numpy as np
import pandas as pd
import random
import scipy as sp
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from itertools import islice
from IPython.display import clear_output
import pytry

In [ ]:
class PatternInterpolationTrial(pytry.Trial):
    def params(self):
        self.param('number of dimensions', n_dims=2),
        self.param('length of pattern', len_train=10),
        self.param('number of epochs', n_epoch=1),
        self.param('seed', p_seed=1),
        
    def evaluate(self, param): #function to fill in missing data points by interpolating
        import nengo
        data = pd.read_csv("multidata.csv", low_memory=False)

        x = np.array(data.iloc[:,11:195]).astype(float) #array of data for purple child (points in space for each frame)
        labs = np.array(data.iloc[:,443]).astype(str) #array of labels (purple child annotations, engagement) 218/443
        a = []
        for i in range(x.shape[1]):
            y = pd.Series(x[:,i])
            if i == 180:               # add these three lines
                y[y>0]-=np.pi*2   # add these three lines
                y += np.pi            # add these three lines
            z = y.interpolate(limit_direction='both')
            a.append(z)
        a = pd.DataFrame(a)
        a = a.dropna()
        a = np.array(a).T
        
        n_dims=2 
        seed=1

        dt = 0.001
        target_dt = 1.0/30
        D = n_dims # param.n_dims
        classify_score = {}
        accuracy = {}
        
        def extract_pattern(start, end, target_dt): #function to extract consecutive frames with the same label 
            pattern = np.array(a[start:end,:]).astype(float)
            frames = np.array(data.iloc[start:end,9]).astype(int)
            timestamp = np.array(data.iloc[start:end,0]).astype(str)

            good_indices = frames != -1
            frames = frames[good_indices]
            pattern = pattern[good_indices]

            fps = 30.0
            t_sample = (frames - frames[0])/fps

            t = np.arange(int(t_sample[-1]/target_dt))*target_dt

            result = []
            for i in range(pattern.shape[1]):       
                p = np.interp(t, t_sample, pattern[:,i])
                result.append(p)
            result = np.array(result).T

            return timestamp, result

        start=[]
        start.append(0)
        end=[]
        label = []
        for i in range(1, (len(labs)-1)):
            if labs[i]!=labs[i-1]:
                start.append(i)
            if labs[i]!=labs[i+1]:
                end.append(i)
                label.append(labs[i])

        t_noplay=[]
        p_noplay=[]
        t_goal=[]
        p_goal=[]
        t_aim=[]
        p_aim=[]

        for i in range(1,(len(start)-1)):
            try:
                if label[i]==('noplay'):
                    ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
                    t_noplay.append(ts)
                    p_noplay.append(pi)
                if label[i]==('goaloriented'):
                    ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
                    t_goal.append(ts)
                    p_goal.append(pi)
                if label[i]==('aimless'):
                    ts, pi = extract_pattern(start[i], end[i], target_dt=target_dt)
                    t_aim.append(ts)
                    p_aim.append(pi)
            except IndexError:
                print('empty pattern')
        
############ randomly split data 80/20 into training and testing sets ############ 
        i=0
        goal = []
        noplay = []
        aim = []
        for entry in t_goal:
            goal.append([t_goal[i],p_goal[i]])
            i = i+1
        i=0
        for entry in t_noplay:
            noplay.append([t_noplay[i],p_noplay[i]])
            i = i+1
        i=0
        for entry in t_aim:
            aim.append([t_aim[i],p_aim[i]])
            i = i+1

        p_goal2 = random.sample(goal, len(goal))
        p_noplay2 = random.sample(noplay, len(noplay))
        p_aim2 = random.sample(aim, len(aim))

        goal_train_ts = np.array(p_goal2[:(int(len(p_goal2)*0.8))]) 
        goal_test_ts = np.array(p_goal2[(int(len(p_goal2)*0.8)):]) 
        goal_train_list=list(goal_train_ts[:,1])
        goal_test_list=list(goal_test_ts[:,1])
        goal_train_time=list(goal_train_ts[:,0])
        goal_test_time=list(goal_test_ts[:,0])

        noplay_train_ts = np.array(p_noplay2[:(int(len(p_noplay2)*0.8))]) 
        noplay_test_ts = np.array(p_noplay2[(int(len(p_noplay2)*0.8)):]) 
        noplay_train_list=list(noplay_train_ts[:,1])
        noplay_test_list=list(noplay_test_ts[:,1])
        noplay_train_time=list(noplay_train_ts[:,0])
        noplay_test_time=list(noplay_test_ts[:,0])
        
        for p in noplay_train_list:
            if p.size > 0:
                pass
            else:
                noplay_train_list.remove(p)

        for p in noplay_test_list:
            if p.size >0:
                pass
            else:
                noplay_test_list.remove(p)

        train_all = np.vstack(goal_train_list+noplay_train_list)
        pca_model = PCA(n_components=D).fit(train_all)

        train_time_all = np.hstack(goal_train_time+noplay_train_time)
        test_time_all = np.hstack(goal_test_time+noplay_test_time)

        goal_train_pca = np.vstack([pca_model.transform(p) for p in goal_train_list])
        noplay_train_pca = np.vstack([pca_model.transform(p) for p in noplay_train_list])
        goal_test_pca = np.vstack([pca_model.transform(p) for p in goal_test_list])
        noplay_test_pca = np.vstack([pca_model.transform(p) for p in noplay_test_list])

        T_train = param.len_train   # number of seconds to train on for each class
        T_test = 100    # number of seconds to test on for each class
        
        goal_train_time2 = [item for sublist in goal_train_time for item in sublist]
        noplay_train_time2 = [item for sublist in noplay_train_time for item in sublist]

        goal_test_time2 = [item for sublist in goal_test_time for item in sublist]
        noplay_test_time2 = [item for sublist in noplay_test_time for item in sublist]

        N_frames = int(T_train*30)
        training = np.vstack([goal_train_pca[:N_frames], noplay_train_pca[:N_frames]])
        train_timestamps = np.hstack([goal_train_time2[:N_frames], noplay_train_time2[:N_frames]])
        assert len(training) == N_frames*2

        N_frames = int(T_test*30)
        testing = np.vstack([goal_test_pca[:N_frames], noplay_test_pca[:N_frames]])
        test_timestamps = np.hstack([goal_test_time2[:N_frames], noplay_test_time2[:N_frames]])
        assert len(testing) == N_frames*2
        
############ TRAINING WITH 80% ############ 
        print('Testing with 80% Training Data')
    
        num_epochs = param.n_epoch
        batch_size = int(T_train*30)
        batch = 0
        theta = 15

        for epoch in range(num_epochs):
            training = np.vstack([goal_train_pca[batch:int(batch+batch_size)], noplay_train_pca[batch:int(batch+batch_size)]])
            batch += batch_size

            net = nengo.Network(seed=seed)#param.seed)
            with net:
                rw = []
                for i in range(D):
                    process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                               presentation_time=1.0/30)
                    rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                              process=process, 
                                                              neuron_type=nengo.Direct()))


                node_pool = nengo.Node(None, size_in=rw[0].state.size_out*D)

                start = 0
                for r in rw:
                    nengo.Connection(r.state, node_pool[start:start+r.state.size_out])
                    start += r.state.size_out



                stim = nengo.Node(nengo.processes.PresentInput(training, presentation_time=1.0/30))
                assert stim.size_out == D
                for i in range(D):
                    nengo.Connection(stim[i], rw[i].input, synapse=None)

                p_node_pool = nengo.Probe(node_pool, sample_every = 0.1)


            sim = nengo.Simulator(net)
            with sim:
                sim.run(T_train*2)  


            pool_model = nengo.Network()
            with pool_model:
                pool = nengo.Ensemble(n_neurons=3000, dimensions=node_pool.size_out,
                                      neuron_type=nengo.LIFRate(), seed=seed)
            pool_sim = nengo.Simulator(pool_model)

            import nengo.utils.ensemble

            _, a = nengo.utils.ensemble.tuning_curves(pool, pool_sim, inputs=sim.data[p_node_pool])

            if epoch == 0:
                a_goal = a[:(int(len(a)/2))]
                a_noplay = a[(int(len(a)/2)):]
            else:
                a_goal = np.concatenate((a_goal, a[:(int(len(a)/2))]))
                a_noplay = np.concatenate((a_noplay, a[(int(len(a)/2)):]))

        a_out = np.vstack([a_goal, a_noplay])


        N = int((len(a_out))/2) #int(T_train*1000)
        target = np.hstack([np.ones(N), -np.ones(N)]).reshape(-1, 1)
        dec, info = nengo.solvers.LstsqL2(reg=0.1)(a_out, target)
        
        v = np.dot(a_out, dec)
        classify1 = np.isclose(v[:N], 1, atol=0.5)
        classify2 = np.isclose(v[N:], -1, atol=0.5)
        classify = np.append(classify1, classify2)
        score_train = np.mean(classify)

        classify_score[0]=score_train

        train_timestamps = train_timestamps[0::3]

        output_train = {}
        for i in range(len(v)):
            output_train[i]={"time":train_timestamps2[i], "out_train":v[i]}

        for j in range(len(output_train)):        
            key_j = 'output_train{}'.format("%04d" % j)  
            output_train[key_j] = output_train.pop(j) 

#        accuracy = dict(islice(enumerate(v), None, None, 100)) 
#        accuracy_train={}
#        for i in range(len(accuracy)):
#            accuracy_train[i]=accuracy[i*100]

#        for j in range(len(accuracy_train)):        
#            key_j = 'accuracy_train{}'.format("%04d" % j)  
#            accuracy_train[key_j] = accuracy_train.pop(j)  

############ TESTING WITH 20% ############ 
        print('Testing with 20% Testing Data')
    
        test_net = nengo.Network(seed=seed)#param.seed)
        with test_net:
            rw = []
            for i in range(D):
                process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                           presentation_time=1.0/30)
                rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                          process=process, 
                                                          neuron_type=nengo.Direct()))


            pool = nengo.Ensemble(n_neurons=3000, dimensions=node_pool.size_out,
                                  neuron_type=nengo.LIFRate(), seed=seed)

            start = 0
            for r in rw:
                nengo.Connection(r.state, pool[start:start+r.state.size_out])
                start += r.state.size_out



            stim = nengo.Node(nengo.processes.PresentInput(testing, presentation_time=1.0/30))
            assert stim.size_out == D
            for i in range(D):
                nengo.Connection(stim[i], rw[i].input, synapse=None)

            p_stim = nengo.Probe(stim)

            result = nengo.Node(None, size_in=1)
            nengo.Connection(pool.neurons, result, transform=dec.T, synapse=None)

            p_test_result_sample = nengo.Probe(result, sample_every = 0.1)


        test_sim = nengo.Simulator(test_net)
        test_sim.run(T_test*2)
        
        N = int(T_test*10)

        v = test_sim.data[p_test_result_sample]
        classify1 = np.isclose(v[:N], 1, atol=0.5)
        classify2 = np.isclose(v[N:], -1, atol=0.5)
        classify = np.append(classify1, classify2)
        score_test = np.mean(classify)

        classify_score[1]=score_test
            
        test_timestamps = test_timestamps[0::3]

        output_test = {}
        for i in range(len(v)):
            output_test[i]={"time":test_timestamps2[i], "out_test":v[i]}

        for j in range(len(output_test)):        
            key_j = 'output_test{}'.format("%04d" % j)  
            output_test[key_j] = output_test.pop(j) 

#        accuracy = dict(islice(enumerate(v), None, None, 100)) 
#        accuracy_test={}
#        for i in range(len(accuracy)):
#            accuracy_test[i]=accuracy[i*100]

#        for j in range(len(accuracy_test)):        
#            key_j = 'accuracy_test{}'.format("%04d" % j)  
#            accuracy_test[key_j] = accuracy_test.pop(j) 

############ TESTING WITH AIMLESS PATTERNS ############ 
        print('Testing with Aimless Data')
    
        p_aim = random.sample(p_aim, len(p_aim))

        aim_train_pca = np.vstack([pca_model.transform(p) for p in p_aim])

        #A_len = int(len(aim_train_pca)/30) #Testing on full aimless data
        #print(A_len)

        N_frames = T_test*30

        test_aim = np.vstack([aim_train_pca[:N_frames]])
        assert len(test_aim) == N_frames
        
        aim_test_net = nengo.Network(seed=seed)#param.seed)
        with aim_test_net:
            rw = []
            for i in range(D):
                process = nengo.processes.PresentInput(np.hstack([goal_train_pca[:,i], noplay_train_pca[:,i]]), 
                                                           presentation_time=1.0/30)
                rw.append(nengolib.networks.RollingWindow(theta=theta, n_neurons=3000, 
                                                          process=process, 
                                                          neuron_type=nengo.Direct()))


            pool = nengo.Ensemble(n_neurons=3000, dimensions=node_pool.size_out,
                                  neuron_type=nengo.LIFRate(), seed=seed)

            start = 0
            for r in rw:
                nengo.Connection(r.state, pool[start:start+r.state.size_out])
                start += r.state.size_out



            stim = nengo.Node(nengo.processes.PresentInput(test_aim, presentation_time=1.0/30))
            assert stim.size_out == D
            for i in range(D):
                nengo.Connection(stim[i], rw[i].input, synapse=None)

            p_stim = nengo.Probe(stim)

            result = nengo.Node(None, size_in=1)
            nengo.Connection(pool.neurons, result, transform=dec.T, synapse=None)

            p_aim_result_sample = nengo.Probe(result, sample_every=0.1)


        aim_test_sim = nengo.Simulator(aim_test_net)
        aim_test_sim.run(T_test)
        
        v = aim_test_sim.data[p_aim_result_sample]
        classify = np.isclose(v, 0, atol=0.5)
        score_aim = np.mean(classify)

        classify_score[2]=score_aim
        
        aim_timestamps = aim_timestamps[0::3]

        output_aim = {}
        for i in range(len(v)):
            output_aim[i]={"time":aim_timestamps[i], "out_aim":v[i]}

        for j in range(len(output_aim)):        
            key_j = 'output_aim{}'.format("%04d" % j)  
            output_aim[key_j] = output_aim.pop(j) 

#        accuracy = dict(islice(enumerate(v), None, None, 100)) 
#        accuracy_aim={}
#        for i in range(len(accuracy)):
#            accuracy_aim[i]=accuracy[i*100]

#        for j in range(len(accuracy_aim)):        
#            key_j = 'accuracy_aim{}'.format("%04d" % j)  
#            accuracy_aim[key_j] = accuracy_aim.pop(j) 
            
############ SAVE DATA ############ 
        classify_score['classify_train'] = classify_score.pop(0)
        classify_score['classify_test'] = classify_score.pop(1)
        classify_score['classify_aim'] = classify_score.pop(2)
        
        #network_output = {**output_train, **output_test, **output_aim}
        accuracy_output = {**accuracy_train, **accuracy_test, **accuracy_aim}
    
        results = {**classify_score, **accuracy_output}
        return results

In [ ]:
PatternInterpolationTrial().run(len_train=500, n_epoch=1, data_dir='theta15')

In [ ]:
for seed in range(20):
    PatternInterpolationTrial().run(seed=seed, p_seed=seed, len_train=10, n_epoch=1, data_dir='theta15')
    clear_output()